# Proyecto 1

In [0]:
from pyspark.sql import functions as f

## 1 - Carga de Datos

### SECOP II - Contratos Electrónicos
Información de los contratos registrados en SECOP II desde su lanzamiento

In [0]:
# Listar archivos para ver que tenemos en el directorio SECOP
dbutils.fs.ls("wasbs://sid@uniandesyjt.blob.core.windows.net/secop")

Para leer el dataset secop de la manera más rápida posible es necesario definir el esquema. 
Sabemos de antemano que el formato es json.
Inicialmente inferimos el esquema con un muestreo del 10%.

In [0]:
df_secop_raw = spark.read.format("json")\
    .option("inferSchema", "true")\
    .option("samplingRatio", 0.1)\
    .load("wasbs://sid@uniandesyjt.blob.core.windows.net/secop/")
df_secop_raw.show(truncate=False)

Visualicemos el esquema del dataframe secop

In [0]:
df_secop_raw.printSchema()

Una vez conocemos el tipo de dato de cada columna, procedemos a definir el esquema y usarlo durante la lectura del dataset.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType

# Define proper schema based on what we can see
schema_secop = StructType([
    StructField("anno_bpin", StringType(), True),
    StructField("anno_firma", IntegerType(), True),
    StructField("ciudad", StringType(), True),
    StructField("código_bpin", StringType(), True),
    StructField("departamento", StringType(), True),
    StructField("documento_proveedor", StringType(), True),
    StructField("duración_del_contrato", StringType(), True),
    StructField("entidad_centralizada", StringType(), True),
    StructField("estado_contrato", StringType(), True),
    StructField("fecha_de_fin_del_contrato", StringType(), True),
    StructField("fecha_de_firma", StringType(), True),
    StructField("fecha_de_inicio_del_contrato", StringType(), True),
    StructField("modalidad_de_contratacion", StringType(), True),
    StructField("objeto_del_contrato", StringType(), True),
    StructField("orden", StringType(), True),
    StructField("origen_de_los_recursos", StringType(), True),
    StructField("proveedor_adjudicado", StringType(), True),
    StructField("rama", StringType(), True),
    StructField("sector", StringType(), True),
    StructField("tipo_de_contrato", StringType(), True),
    StructField("tipodocproveedor", StringType(), True),
    StructField("ultima_actualizacion", StringType(), True),
    StructField("urlproceso", StringType(), True),
    StructField("valor_del_contrato", LongType(), True),
    StructField("valor_pagado", StringType(), True)
])

# Read with proper schema
df_secop = spark.read.format("json")\
    .schema(schema_secop)\
    .load("wasbs://sid@uniandesyjt.blob.core.windows.net/secop/")

df_secop.printSchema()
df_secop.show(5)

In [0]:
display(df_secop)

Verifiquemos que en la lectura hemos obtenido los registros de todos los años.

In [0]:
df_secop.select("anno_firma").distinct().orderBy("anno_firma").show()

Una visualización de algunas filas de secop:

In [0]:
df_secop.show(5)

Tenemos 4'447,723 registros en secop

In [0]:
print(f"Total rows secop: {df_secop.count()}")

## DNP-proyectos_datos_basicos
Mapa Inversiones. Datos generales y relevantes de un proyecto de inversión pública en Colombia.

Tenemos un archivo bpin.csv.gz

In [0]:
# Listemos los archivos en el directorio bpin
dbutils.fs.ls("wasbs://sid@uniandesyjt.blob.core.windows.net/bpin")

Para leer el dataset bpin de la manera más rápida vamos a inferir el esquema leyendo el archivo en formato csv y usando los encabezados para nombrar las columnas del dataframe.

In [0]:
df_proyectos_raw = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .option("samplingRatio", 0.1)\
    .load("wasbs://sid@uniandesyjt.blob.core.windows.net/bpin/bpin.csv.gz")

df_proyectos_raw.printSchema()

In [0]:
# Definir el schema
schema_bpin = StructType([
    StructField("Bpin", StringType(), True),
    StructField("NombreProyecto", StringType(), True),
    StructField("ObjetivoGeneral", StringType(), True),
    StructField("EstadoProyecto", StringType(), True),
    StructField("Horizonte", StringType(), True),
    StructField("Sector", StringType(), True),
    StructField("EntidadResponsable", StringType(), True),
    StructField("ProgramaPresupuestal", StringType(), True),
    StructField("TipoProyecto", StringType(), True),
    StructField("PlanDesarrolloNacional", StringType(), True),
    StructField("ValorTotalProyecto", DoubleType(), True),
    StructField("ValorVigenteProyecto", DoubleType(), True),
    StructField("ValorObligacionProyecto", DoubleType(), True),
    StructField("ValorPagoProyecto", DoubleType(), True),
    StructField("SubEstadoProyecto", StringType(), True),
    StructField("CodigoEntidadResponsable", StringType(), True),
    StructField("TotalBeneficiario", IntegerType(), True)
])

df_proyectos = spark.read.format("csv") \
    .option("header", "true") \
    .schema(schema_bpin) \
    .load("wasbs://sid@uniandesyjt.blob.core.windows.net/bpin/bpin.csv.gz")




Tenemos 498,306 filas en nuestro dataframe bpin

In [0]:
print(f"Total rows bpin: {df_proyectos.count()}")

Visualicemos algunas filas de nuestro dataframe bpin

In [0]:
df_proyectos.show(5)

In [0]:
display(df_proyectos)

In [0]:
display(df_secop)

In [0]:
display(df_proyectos)

## 2 - Identifique los 10 proveedores que han tenido el mayor valor de contratos durante el año 2024. 

In [0]:
from pyspark.sql import functions as f

In [0]:
contratos = (
    df_secop.select(
        f.upper(f.col("proveedor_adjudicado")).alias("proveedor"),
        f.col("objeto_del_contrato").alias("objeto"),
        f.col("anno_firma").alias("anno"),
        f.col("código_bpin").alias("bpin"),
        f.col("valor_del_contrato").cast("double").alias("valor_contrato"),
        f.regexp_replace(f.col("valor_pagado"), r"[$,]", "")
        .cast("double")
        .alias("valor_pagado")
        )
    .withColumn("valor_contrato_sin_pagar", f.col("valor_contrato") - f.col("valor_pagado"))
    )
contratos.show(10)

In [0]:
top_contratos = (
    contratos.select("proveedor", "anno")
    .where(f.col("anno") == 2024)
    .groupBy("proveedor").count().orderBy("count", ascending=False)
    )

In [0]:
top_contratos.show(10, truncate=False)

## 3 - Identifique los 10 Proyectos de inversión que han tenido el mayor valor de contratos sin pagar (valor del contrato menos el valor pagado). La respuesta debe incluir el nombre de los proyectos y el valor sin pagar.

In [0]:
proyectos = (
    df_proyectos.select(
        f.upper(f.col("NombreProyecto")).alias("nombre_proyecto"),
        f.col("Bpin").alias("bpin"),
        f.col("ValorTotalProyecto"),
        f.col("ValorPagoProyecto")
    )
)
proyectos.show(5)

In [0]:
proyecto_contrato = (
    proyectos.join(contratos, on="bpin", how="inner")
    .select("objeto","nombre_proyecto", "anno", "bpin", "valor_contrato", "valor_pagado", "valor_contrato_sin_pagar","ValorTotalProyecto", "ValorPagoProyecto")
    .groupBy("nombre_proyecto","ValorTotalProyecto")
    .agg(
        f.sum(f.col("valor_contrato_sin_pagar")).alias("total_contratos_sin_pagar"),
        f.sum(f.col("valor_contrato")).alias("total_valor_contratos"),
        f.sum(f.col("valor_pagado")).alias("total_valor_pagado"),
        f.count("*").alias("numero_contratos")
    )
    .withColumn("%_ejecucion_contratos", f.round((f.try_divide(f.col("total_valor_pagado"),f.col("total_valor_contratos")))*100, 2))
#   .withColumn("diferencia_proyecto_contratos", f.col("ValorTotalProyecto") - f.col("total_valor_contratos"))
    .orderBy("total_contratos_sin_pagar", ascending=False)
    .drop("ValorTotalProyecto", "total_valor_contratos", "total_valor_pagado")
    .limit(10)
)
proyecto_contrato.show(10)

In [0]:
display(proyecto_contrato)

## 4 -  Identifique para cada año el top 5 de los proveedores con mayor valor de contratos. La respuesta debe incluir el nombre de los proveedores, el valor de sus contratos y su posición en el top.

In [0]:
from pyspark.sql import Window
ventana = Window.partitionBy("anno").orderBy(f.desc("total_contratos"))

display(
    contratos.select(
        f.col("proveedor"),
        f.col("anno"),
        f.col("valor_contrato")
    )
    .groupBy("anno","proveedor")
    .agg(f.sum("valor_contrato").alias("total_contratos"))
    .withColumn("rank", f.row_number().over(ventana))
    .filter(f.col("rank") <= 5)
    .orderBy("anno", "rank")
)

## 5 - Identifique el número de proveedores que firmaron contratos en el 2024 y NO firmaron contratos en el 2020. 

In [0]:
contratos_2020 = contratos.where(f.col("anno") == 2020)
proveedores_2020 = (
    contratos_2020.groupBy("proveedor").count().orderBy("proveedor")
    .drop("count")
    )
display(proveedores_2020.limit(50))
display(proveedores_2020.count())

In [0]:
contratos_2024 = contratos.where(f.col("anno") == 2024)
proveedores_2024 = (
    contratos_2024.groupBy("proveedor").count().orderBy("proveedor")
    .drop("count")
    )
display(proveedores_2024.limit(50))
display(proveedores_2024.count())

In [0]:

proveedores_2024_not_2020 = (
    proveedores_2024.join(proveedores_2020, on="proveedor", how="left_anti")
    .orderBy("proveedor")
)
display(proveedores_2024_not_2020.limit(50))
display(proveedores_2024_not_2020.count())

Por ejemplo #NARRARELFUTURO fue proveedor de contratos en 2024 pero no lo fue en 2020.

- \*TECHNICAL ADJUSTMENTS JASS\*  
- (SECRETARÍA DISTRITAL DE INTEGRACIÓN SOCIAL)  
contrataron en 2020 y en 2024. Por lo tanto no se visualizan



## 6 - Calcule el valor promedio de los contratos de “Prestación de servicios” y una aproximación de la mediana (el percentil 50). 

In [0]:
# 1) Filtrar contratos de tipo "Prestación de servicios" (tolerante a acentos/mayúsculas)
df_ps = (
    df_secop
    .select("tipo_de_contrato", F.col("valor_del_contrato").cast("double").alias("valor_del_contrato"))
    .dropna(subset=["tipo_de_contrato", "valor_del_contrato"])
    .withColumn("tipo_lc", F.lower(F.col("tipo_de_contrato")))
    .filter(
        F.col("tipo_lc").like("%prestación de servicios%") |
        F.col("tipo_lc").like("%prestacion de servicios%")
    )
    .filter(F.col("valor_del_contrato") > 0)
)

# 2) Muestreo para acelera
sample_fraction = 0.20
df_ps_fast = df_ps.sample(withReplacement=False, fraction=sample_fraction, seed=42)

# 3) Promedio
promedio = df_ps_fast.agg(F.avg("valor_del_contrato").alias("promedio")).first()["promedio"]

# 4) Mediana aproximada 
p50 = df_ps_fast.selectExpr("approx_percentile(valor_del_contrato, 0.5) as p50").first()["p50"]

total_filtrados = df_ps.count()
total_muestra = df_ps_fast.count()

print(f"Contratos 'Prestación de servicios' filtrados: {total_filtrados:,}")
print(f"Tamaño de la muestra: {total_muestra:,} (fraction={sample_fraction})")
print(f"Promedio (aprox): {promedio:,.2f}")
print(f"Mediana (p50 aprox): {p50:,.2f}")


## 7. Cuál fue el año en el que firmaron contratos un mayor número de proveedores distintos. Construya esta respuesta de manera que se minimice el tiempo de cálculo, aunque se sacrifique precisión.

## 8. Cuáles son las 20 palabras más comunes en los objetos contractuales del 20% de los contratos más altos del 2020. Para esta respuesta omita [Stopwords](https://pypi.org/project/stop-words/) puntuaciones y no diferencie entre mayúsculas y minúsculas.